获得代理ip

In [ ]:
import requests
import re
import time
import json
import telnetlib

def ip_test(ip,port):
    try:
        telnetlib.Telnet(ip,port,timeout=1.5)
    except:
        return False
    else:
        return True

def url_parse(page):
    url = "https://www.kuaidaili.com/free/intr/{}/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36"
    }
    resp = requests.get(url.format(page), headers=headers)
    ips = re.findall('"IP">(.+?)</td>.+?"PORT">(.+?)</td>', resp.content.decode(), re.S)
    return ips

def urls_parse():
    ips = []
    for page in range(1,10):
        ips += url_parse(page)
        print(ips)
        time.sleep(1)
    return ips
# ret = urls_parse()
# print(ret)

def proxies_test(ips):
    ip_ok = []
    for ip in ips:
        if ip_test(ip[0],ip[1]):
            ip_ok.append(ip)
            print(ip)
    ip_ok = ["%s:%s"%(ip,port) for ip,port in ip_ok]

    return ip_ok

# ip_ok = proxies_test(urls_parse())
# print("result:",ip_ok)
# ips = ['14.20.235.201:9797', '124.205.155.150:9090', '218.66.253.146:8800', '14.155.115.157:9000']
def save_ips(ips):
    with open("json/ips.json","w+") as f:
        f.write(json.dumps(ips))


if __name__ == "__main__":
    ips = urls_parse()
    ips = proxies_test(ips)
    ret = save_ips(ips)

# print(json.dumps(ips))

使用代理

In [ ]:
import requests
import json
import random

url = "http://httpbin.org/get"
url2 = "http://icanhazip.com/"
url3 = "https://www.baidu.com"
headers = [{"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36"},
    {"User-Agent": "Mozilla/5.0 (Linux; Android 5.0; SM-G900P Build/LRX21T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Mobile Safari/537.36"}
]

def get_ips():
    with open("json/ips.json","r") as f:
        ips = f.read()
    ips = json.loads(ips)
    return ips

def proxies_test2():
    ips = get_ips()
    ip = "http://" + random.choice(ips)
    proxies = {
        "http": ip,
        "https": ip
    }
    resp = requests.get(url2, headers=random.choice(headers), proxies=proxies)
    print(ip,resp.content.decode())

def proxies_test3():
    ips = get_ips()
    for ip in  ips:
        proxies = {
            "http": ip,
            "https": ip
        }
        try:
            resp = requests.get(url2, headers=random.choice(headers), proxies=proxies)
        except:
            print("%s fail"%ip)
        else:
            print("ip success %s"%ip)
            print(resp.content.decode())
            
if __name__ == "__main__":
    ret = get_ips()
    proxies_test3()

无头浏览器

chromedriver_mac64.zip https://chromedriver.storage.googleapis.com/index.html?path=96.0.4664.45/
pip install selenium

In [ ]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.support.select import Select   # 对 select 的处理
from selenium.webdriver.chrome.options import Options  # 无头的模块导入
import re

# 准备好参数配置   不用记  要用的时候 到时候粘过来
opt = Options()
opt.add_argument('--headless')
opt.add_argument('disbale-gpu')  # 不显示

web = Chrome('/usr/local/bin/chromedriver', options=opt)  # 把参数配置设置到浏览器中

# ids = ['tree_' + str(i) + '_span' for i in range(1, 42)]  # 42
# url = r'http://hn.12348.gov.cn/fxmain/subpage/legalpublicity/fx_regulations.html?objId=1394935113142247424'  # 《85普法读本》

# ids = ['tree_'+str(i)+'_span' for i in range(1,8)]
# url = 'http://hn.12348.gov.cn/fxmain/subpage/legalpublicity/fx_regulations.html?objId=1399198957574299648' #党内法规知识

ids = ['tree_' + str(i) + '_span' for i in range(1, 36)]
url = 'http://hn.12348.gov.cn/fxmain/subpage/legalpublicity/fx_regulations.html?objId=1399192447649587200'  # 2021普法读本

web.get(url)
# 怎样拿到页面代码（ 经过数据加载以及js执行之后的结果的html内容）
# print(web.page_source) # print(web.page_source)

fp = open('搜法宝.txt', 'a+')
# 定位到导航链接 a：
for a in ids:
    link = web.find_element_by_xpath('//*[@id="{}"]'.format(a))
    print('-' * 60, '\n', link.text, '\n')
    fp.write('-' * 60 + '\n' + link.text + '\n')
    link.click()
    time.sleep(1)

    content = web.find_element_by_class_name("s_flzs_fnei")
    outHTML = content.get_attribute('innerHTML')
    outHTML = outHTML.replace('<p', '\n<p')
    pattern = re.compile(r'<[^>]+>', re.S)
    print(pattern.sub('', outHTML).replace('&nbsp;', ''))
    fp.write(pattern.sub('', outHTML).replace('&nbsp;', ''))

web.close()
fp.close()
print('\n' * 4, '运行完毕！')
